In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, classification_report, accuracy_score

from sklearn.pipeline import FeatureUnion

In [2]:
def get_data():
    file_name = './LanguageSamples.txt'
    rawdata = open(file_name, 'r')
    lines = rawdata.readlines()
    return lines
#     lines = lines[1:] #get rid of "header"
#     spam_train = lines[0:1000]
#     ham_train = lines[1002:2002]
#     test_mix = lines[2004:]
#     return (spam_train, ham_train, test_mix)

In [3]:
data_lines = get_data()

In [8]:
data_lines[0:13]

['\ufeff  <pre lang="XML">\n',
 '&lt;?xml version="1.0"?&gt;\n',
 '&lt;DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18"&gt;\n',
 '  &lt;SQLInstance&gt;(localdb)\\v11.0&lt;/SQLInstance&gt;\n',
 '  &lt;PageBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\PageBlobRoot&lt;/PageBlobRoot&gt;\n',
 '  &lt;BlockBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\BlockBlobRoot&lt;/BlockBlobRoot&gt;\n',
 '  &lt;LogPath&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\Logs&lt;/LogPath&gt;\n',
 '  &lt;LoggingEnabled&gt;false&lt;/LoggingEnabled&gt;\n',
 '&lt;/DevelopmentStorage&gt;</pre>\n',
 '\n',
 '\n',
 '\n',
 '<pre lang="Swift">\n']

In [ ]:
#read it all in as a string, split it on <pre; replae \n with ' ' replace &lt $gt with < > 
#get rid of \n lines

In [ ]:
# for birds - one hot encoding the tags

In [ ]:
/<pre\s?(.*)>(.*)<\/pre>/

In [ ]:
#this works
<pre[\s\S]*?<\/pre>

In [9]:
import re

In [34]:
regex = '<pre[\s\S]*?<\/pre>'
p = re.compile(regex)

In [35]:
samples = data_lines[0:26]

In [36]:
samples

['\ufeff  <pre lang="XML">\n',
 '&lt;?xml version="1.0"?&gt;\n',
 '&lt;DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18"&gt;\n',
 '  &lt;SQLInstance&gt;(localdb)\\v11.0&lt;/SQLInstance&gt;\n',
 '  &lt;PageBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\PageBlobRoot&lt;/PageBlobRoot&gt;\n',
 '  &lt;BlockBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\BlockBlobRoot&lt;/BlockBlobRoot&gt;\n',
 '  &lt;LogPath&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\Logs&lt;/LogPath&gt;\n',
 '  &lt;LoggingEnabled&gt;false&lt;/LoggingEnabled&gt;\n',
 '&lt;/DevelopmentStorage&gt;</pre>\n',
 '\n',
 '\n',
 '\n',
 '<pre lang="Swift">\n',
 '@objc func handleTap(sender: UITapGestureRecognizer) {\n',
 '    if let tappedSceneView = sender.view as? ARSCNView {\n',
 '        let tapLocationInView = sender.location(in: tappedSceneView)\n',
 '        let planeHitTest = tappedSceneView

In [37]:
all_samples = ''.join(samples)

In [38]:
print(all_samples)

﻿  <pre lang="XML">
&lt;?xml version="1.0"?&gt;
&lt;DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18"&gt;
  &lt;SQLInstance&gt;(localdb)\v11.0&lt;/SQLInstance&gt;
  &lt;PageBlobRoot&gt;C:\Users\Carl\AppData\Local\DevelopmentStorage\PageBlobRoot&lt;/PageBlobRoot&gt;
  &lt;BlockBlobRoot&gt;C:\Users\Carl\AppData\Local\DevelopmentStorage\BlockBlobRoot&lt;/BlockBlobRoot&gt;
  &lt;LogPath&gt;C:\Users\Carl\AppData\Local\DevelopmentStorage\Logs&lt;/LogPath&gt;
  &lt;LoggingEnabled&gt;false&lt;/LoggingEnabled&gt;
&lt;/DevelopmentStorage&gt;</pre>



<pre lang="Swift">
@objc func handleTap(sender: UITapGestureRecognizer) {
    if let tappedSceneView = sender.view as? ARSCNView {
        let tapLocationInView = sender.location(in: tappedSceneView)
        let planeHitTest = tappedSceneView.hitTest(tapLocationInView,
            types: .existingPlaneUsingExtent)
        if !planeHitTest.isEmpty {
            a

In [39]:
test = '<pre>test</pre>more<pre>test1</pre'

In [40]:
matches = p.findall(all_samples)

In [41]:
print(matches)

['<pre lang="XML">\n&lt;?xml version="1.0"?&gt;\n&lt;DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18"&gt;\n  &lt;SQLInstance&gt;(localdb)\\v11.0&lt;/SQLInstance&gt;\n  &lt;PageBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\PageBlobRoot&lt;/PageBlobRoot&gt;\n  &lt;BlockBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\BlockBlobRoot&lt;/BlockBlobRoot&gt;\n  &lt;LogPath&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\Logs&lt;/LogPath&gt;\n  &lt;LoggingEnabled&gt;false&lt;/LoggingEnabled&gt;\n&lt;/DevelopmentStorage&gt;</pre>', '<pre lang="Swift">\n@objc func handleTap(sender: UITapGestureRecognizer) {\n    if let tappedSceneView = sender.view as? ARSCNView {\n        let tapLocationInView = sender.location(in: tappedSceneView)\n        let planeHitTest = tappedSceneView.hitTest(tapLocationInView,\n            types: .existingPlaneUsingExtent)\n        if !plan

In [29]:
for m in matches:
    print(m)

TypeError: '_sre.SRE_Match' object is not iterable

In [42]:
all_found = re.findall(r'<pre[\s\S]*?<\/pre>', all_samples, re.MULTILINE)

In [43]:
all_found

['<pre lang="XML">\n&lt;?xml version="1.0"?&gt;\n&lt;DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18"&gt;\n  &lt;SQLInstance&gt;(localdb)\\v11.0&lt;/SQLInstance&gt;\n  &lt;PageBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\PageBlobRoot&lt;/PageBlobRoot&gt;\n  &lt;BlockBlobRoot&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\BlockBlobRoot&lt;/BlockBlobRoot&gt;\n  &lt;LogPath&gt;C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\Logs&lt;/LogPath&gt;\n  &lt;LoggingEnabled&gt;false&lt;/LoggingEnabled&gt;\n&lt;/DevelopmentStorage&gt;</pre>',
 '<pre lang="Swift">\n@objc func handleTap(sender: UITapGestureRecognizer) {\n    if let tappedSceneView = sender.view as? ARSCNView {\n        let tapLocationInView = sender.location(in: tappedSceneView)\n        let planeHitTest = tappedSceneView.hitTest(tapLocationInView,\n            types: .existingPlaneUsingExtent)\n        if !pla

In [45]:
for m in all_found: 
    print(m)
    print('----------------')

<pre lang="XML">
&lt;?xml version="1.0"?&gt;
&lt;DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18"&gt;
  &lt;SQLInstance&gt;(localdb)\v11.0&lt;/SQLInstance&gt;
  &lt;PageBlobRoot&gt;C:\Users\Carl\AppData\Local\DevelopmentStorage\PageBlobRoot&lt;/PageBlobRoot&gt;
  &lt;BlockBlobRoot&gt;C:\Users\Carl\AppData\Local\DevelopmentStorage\BlockBlobRoot&lt;/BlockBlobRoot&gt;
  &lt;LogPath&gt;C:\Users\Carl\AppData\Local\DevelopmentStorage\Logs&lt;/LogPath&gt;
  &lt;LoggingEnabled&gt;false&lt;/LoggingEnabled&gt;
&lt;/DevelopmentStorage&gt;</pre>
----------------
<pre lang="Swift">
@objc func handleTap(sender: UITapGestureRecognizer) {
    if let tappedSceneView = sender.view as? ARSCNView {
        let tapLocationInView = sender.location(in: tappedSceneView)
        let planeHitTest = tappedSceneView.hitTest(tapLocationInView,
            types: .existingPlaneUsingExtent)
        if !planeHitTest.isEmpty {
  

In [46]:
x = all_found[0]


In [53]:
x = x.replace('&lt;', '<').replace('&gt;', '>').replace('</pre>', '')

In [54]:
print(x)

<pre lang="XML">
<?xml version="1.0"?>
<DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18">
  <SQLInstance>(localdb)\v11.0</SQLInstance>
  <PageBlobRoot>C:\Users\Carl\AppData\Local\DevelopmentStorage\PageBlobRoot</PageBlobRoot>
  <BlockBlobRoot>C:\Users\Carl\AppData\Local\DevelopmentStorage\BlockBlobRoot</BlockBlobRoot>
  <LogPath>C:\Users\Carl\AppData\Local\DevelopmentStorage\Logs</LogPath>
  <LoggingEnabled>false</LoggingEnabled>
</DevelopmentStorage>


In [55]:
splitted = x.split('\n')

In [58]:
indicator = splitted[0]
x1 = splitted[1:]

In [59]:
x1

['<?xml version="1.0"?>',
 '<DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18">',
 '  <SQLInstance>(localdb)\\v11.0</SQLInstance>',
 '  <PageBlobRoot>C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\PageBlobRoot</PageBlobRoot>',
 '  <BlockBlobRoot>C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\BlockBlobRoot</BlockBlobRoot>',
 '  <LogPath>C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\Logs</LogPath>',
 '  <LoggingEnabled>false</LoggingEnabled>',
 '</DevelopmentStorage>']

In [60]:
res = ''.join(x1)

In [61]:
res

'<?xml version="1.0"?><DevelopmentStorage xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2009-03-18">  <SQLInstance>(localdb)\\v11.0</SQLInstance>  <PageBlobRoot>C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\PageBlobRoot</PageBlobRoot>  <BlockBlobRoot>C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\BlockBlobRoot</BlockBlobRoot>  <LogPath>C:\\Users\\Carl\\AppData\\Local\\DevelopmentStorage\\Logs</LogPath>  <LoggingEnabled>false</LoggingEnabled></DevelopmentStorage>'

In [62]:
indicator

'<pre lang="XML">'

In [66]:
#<pre lang="XML">
#re.findall(r'<pre lang="\d">', indicator, re.)

[]

In [74]:
re.findall(r'<pre lang="(.*?)">', indicator, re.MULTILINE)

['XML']

In [75]:
line = re.sub(r'<pre lang="(.*?)">', "", indicator)

In [76]:
line

''